## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.xception.Xception(include_top=False, 
                                             weights='imagenet', 
                                             input_tensor=input_tensor, 
                                             input_shape=None, 
                                             pooling=None)
'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


Using TensorFlow backend.
C:\Users\ggaar\Anaconda3\envs\cvMarathon\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ggaar\Anaconda3\envs\cvMarathon\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ggaar\Anaconda3\envs\cvMarathon\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ggaar\Anaconda3\envs\cv








__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
blo

In [2]:
print('Model深度：', len(model.layers))

Model深度： 132


## 添加層數

In [3]:
x = model.output

'''可以參考Cifar10實作章節'''
x = Convolution2D(32, (3, 3), padding='same', activation='relu', name='convCifar_1')(x)

x = Flatten(name='flatten')(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 135


C:\Users\ggaar\Anaconda3\envs\cvMarathon\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [5]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [7]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 95s 2ms/step - loss: 1.4454 - acc: 0.4852
Epoch 2/100
50000/50000 [==============================] - 89s 2ms/step - loss: 1.0198 - acc: 0.6493
Epoch 3/100
50000/50000 [==============================] - 91s 2ms/step - loss: 0.9000 - acc: 0.6890
Epoch 4/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.8060 - acc: 0.7221
Epoch 5/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.7362 - acc: 0.7458
Epoch 6/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.6666 - acc: 0.7672
Epoch 7/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.6007 - acc: 0.7910
Epoch 8/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.5434 - acc: 0.8121
Epoch 9/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.4887 - acc: 0.8

50000/50000 [==============================] - 91s 2ms/step - loss: 0.0668 - acc: 0.9775
Epoch 76/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0689 - acc: 0.9773: 4 - ETA: 0s - loss: 0.0689 - acc: 0.97
Epoch 77/100
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0689 - acc: 0.9776
Epoch 78/100
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0667 - acc: 0.9779
Epoch 79/100
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0662 - acc: 0.9787
Epoch 80/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.0671 - acc: 0.9775
Epoch 81/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.0633 - acc: 0.9796:
Epoch 82/100
50000/50000 [==============================] - 93s 2ms/step - loss: 0.0701 - acc: 0.9767
Epoch 83/100
50000/50000 [==============================] - 90s 2ms/step - loss: 0.0656 - acc: 0.9781
Epoch 84/100
50000/50000 [============================